In [2]:
import pandas as pd
import numpy as np

In [3]:
pwd

'/Users/edwardrodger/Documents/BARCELONA_22-3/IRONHACK/Course/final_project/final_project_bicing'

In [4]:
ago_18 = pd.read_csv("data/2018_08_Agost_BICING_US.csv")
set_18 = pd.read_csv("data/2018_09_Setembre_BICING_US.csv", sep=';') # all the columns are joined into one here for some reason
oct_18 = pd.read_csv("data/2018_10_Octubre_BICING_US.csv")
nov_18 = pd.read_csv("data/2018_11_Novembre_BICING_US.csv")
des_18 = pd.read_csv("data/2018_12_Desembre_BICING_US.csv")
gen_19 = pd.read_csv("data/2019_01_Gener_BICING_US.csv")
feb_19 = pd.read_csv("data/2019_02_Febrer_BICING_US.csv")
mar_19 = pd.read_csv("data/2019_03_Marc_BICING_US.csv")

In [5]:
frames = [ago_18, set_18, oct_18, nov_18, des_18, gen_19, feb_19, mar_19]
combined_df = pd.concat(frames, ignore_index=True)

In [6]:
combined_df

,error,bikesInUsage,electricalBikesInUsage,mechanicalBikesInUsage,dateTime
0,0,659,14,645,2018-08-01 17:43:27
1,0,666,16,650,2018-08-01 17:45:40
2,0,648,11,637,2018-08-01 18:00:42
3,0,845,13,832,2018-08-01 18:15:40
4,0,889,13,876,2018-08-01 18:30:40
...,...,...,...,...,...
47265,0,3,3,0,2019-03-24 23:34:55
47266,0,3,3,0,2019-03-24 23:39:54
47267,0,3,3,0,2019-03-24 23:44:55
47268,0,3,3,0,2019-03-24 23:49:54


In [7]:
import seaborn as sns
import matplotlib.pyplot as plt 

In [8]:
sns.set_context("poster")
sns.set(rc={"figure.figsize": (12.,6.)})
sns.set_style("whitegrid")

In [9]:
#^^ initial observations: 
# As expected, there is seasonality, although not much detail can be observed here.
#Potential problem - lower amount of data for later months?

In [10]:
sns.set_context("poster")
sns.set(rc={"figure.figsize": (100.,6.)})
sns.set_style("whitegrid")

In [11]:
frames_2 = [ago_18, set_18, oct_18, nov_18, des_18]
df_2 = pd.concat(frames_2, ignore_index=True)

In [12]:
sns.set_context("poster")
sns.set(rc={"figure.figsize": (12.,6.)})
sns.set_style("whitegrid")

In [13]:
#Creating columns for weekends/bank holidays, + morning, afternoon, evening, night

In [15]:
df_2['dateTime'] = pd.to_datetime(df_2['dateTime'])

bank_holidays = ['2018-08-15', '2018-10-12', '2018-11-01', '2018-12-06', '2018-12-08', '2018-12-25']

df_2['is_weekend'] = df_2['dateTime'].dt.dayofweek // 5 == 1
df_2['is_bank_holiday'] = df_2['dateTime'].dt.strftime('%Y-%m-%d').isin(bank_holidays)

In [20]:
df_2['is_weekend_or_bank_holiday'] = df_2['is_weekend'] | df_2['is_bank_holiday']

In [33]:
eve_of_bank_holidays = ['2018-08-14', '2018-10-11', '2018-10-31', '2018-12-05', '2018-12-07', '2018-12-24']

In [37]:
df_2['is_eve_of_weekend_or_bank_holiday'] = (
    (df_2['dateTime'].dt.dayofweek == 4) |
    (df_2['dateTime'].dt.dayofweek == 5) | 
    (df_2['dateTime'].dt.strftime('%Y-%m-%d').isin(eve_of_bank_holidays))
)

#this returns fridays, saturdays, and eves of bank holidays - will be useful to track activity related to nightlife

In [39]:
columns_to_drop = ['is_weekend', 'is_bank_holiday']

In [40]:
df_2 = df_2.drop(columns=columns_to_drop)

In [42]:
#Now adding time of day

In [43]:
morning_range = (6, 12)
afternoon_range = (12, 18)
evening_range = (18, 24)
night_range = (0, 6)

In [44]:
def categorize_time(hour):
    if morning_range[0] <= hour < morning_range[1]:
        return 'Morning'
    elif afternoon_range[0] <= hour < afternoon_range[1]:
        return 'Afternoon'
    elif evening_range[0] <= hour < evening_range[1]:
        return 'Evening'
    else:
        return 'Night'

In [45]:
df_2['time_of_day'] = df_2['dateTime'].dt.hour.apply(categorize_time)

In [46]:
df_2.sample(30)

,error,bikesInUsage,electricalBikesInUsage,mechanicalBikesInUsage,dateTime,is_weekend_or_bank_holiday,is_eve_of_weekend_or_bank_holiday,time_of_day
11268,0,337,4,333,2018-11-02 12:31:38,False,True,Afternoon
14438,0,685,4,681,2018-11-13 20:05:01,False,False,Evening
14241,0,23,0,23,2018-11-13 03:30:04,False,False,Night
1905,0,411,10,401,2018-08-21 14:15:56,False,False,Afternoon
17749,0,255,5,250,2018-12-01 00:00:04,True,True,Night
19637,0,219,3,216,2018-12-08 00:20:06,True,True,Night
6153,0,40,0,40,2018-10-15 06:01:14,False,False,Morning
8651,0,407,7,400,2018-10-24 10:11:30,False,False,Morning
11217,0,479,8,471,2018-11-02 08:16:38,False,True,Morning
2707,0,467,6,461,2018-08-30 16:30:58,False,False,Afternoon


In [47]:
df_2["error"].value_counts()

error
0    25968
Name: count, dtype: int64

In [49]:
df_2 = df_2.drop(columns = "error")

In [51]:
#Ok, now time for the weather API. 
#Goal: return (as accurately as possible, preferably within hour ranges or even 15 mins) 
#1. Temperature and 2. Rain (Y/N) or Precipitation %?

In [58]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 41.38879,
	"longitude": 2.15899,
	"start_date": "2018-08-01",
	"end_date": "2018-12-31",
	"hourly": ["temperature_2m", "rain"],
	"timezone": "Europe/Berlin"
}
responses = openmeteo.weather_api(url, params=params)

In [60]:
response = responses[0]
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 41.37082290649414°E 2.068965435028076°N
Elevation 44.0 m asl
Timezone b'Europe/Berlin' b'CET'
Timezone difference to GMT+0 3600 s


In [63]:
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_rain = hourly.Variables(1).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s"),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["rain"] = hourly_rain

hourly_weather_df = pd.DataFrame(data = hourly_data)

In [64]:
hourly_weather_df

,date,temperature_2m,rain
0,2018-07-31 23:00:00,24.189999,0.0
1,2018-08-01 00:00:00,24.039999,0.0
2,2018-08-01 01:00:00,25.189999,0.0
3,2018-08-01 02:00:00,24.689999,0.0
4,2018-08-01 03:00:00,23.939999,0.0
...,...,...,...
3667,2018-12-31 18:00:00,8.690001,0.0
3668,2018-12-31 19:00:00,7.890000,0.0
3669,2018-12-31 20:00:00,7.090000,0.0
3670,2018-12-31 21:00:00,6.690000,0.0


In [65]:
hourly_weather_df['date'] = pd.to_datetime(hourly_weather_df['date'])

In [69]:
def get_temperature(row):
    hour_category = row['dateTime'].replace(minute=0, second=0, microsecond=0)
    matching_row = hourly_weather_df[hourly_weather_df['date'] == hour_category]
    
    if not matching_row.empty:
        return matching_row.iloc[0]['temperature_2m']
    else:
        return None

In [75]:
def get_precipitation(row):
    hour_category = row['dateTime'].replace(minute=0, second=0, microsecond=0)
    matching_row = hourly_weather_df[hourly_weather_df['date'] == hour_category]
    
    if not matching_row.empty:
        return matching_row.iloc[0]['rain']
    else:
        return None

In [71]:
df_2['temperature'] = df_2.apply(get_temperature, axis=1)

In [76]:
df_2['precipitation'] = df_2.apply(get_precipitation, axis=1)

In [79]:
df_2.sample(20)

,bikesInUsage,electricalBikesInUsage,mechanicalBikesInUsage,dateTime,is_weekend_or_bank_holiday,is_eve_of_weekend_or_bank_holiday,time_of_day,temperature,precipitation
2070,335,3,332,2018-08-23 07:30:58,False,False,Morning,23.639999,0.0
24822,350,2,348,2018-12-27 15:15:11,False,False,Afternoon,13.240001,0.0
13104,304,1,303,2018-11-08 22:26:45,False,False,Evening,12.090000,0.0
20186,21,0,21,2018-12-10 02:15:06,False,False,Night,8.840000,0.0
23568,344,4,340,2018-12-22 21:15:10,True,True,Evening,9.040000,0.0
838,291,3,288,2018-08-10 11:15:36,False,True,Morning,25.939999,0.0
1841,329,0,329,2018-08-20 22:15:55,False,False,Evening,23.289999,0.0
10350,720,7,713,2018-10-30 08:01:39,False,False,Morning,9.090000,0.0
22878,95,3,92,2018-12-20 06:30:10,False,False,Morning,7.140000,0.0
469,387,10,377,2018-08-06 14:45:38,False,False,Afternoon,31.990000,0.0


In [80]:
bicing_final = df_2

In [81]:
bicing_final.to_csv("data/bicing_final.csv", index = True)